In [1]:
import pandas as pd 

**Programda kullanılacak veri setlerini yükledik.**

In [2]:
df_books_ratings = pd.read_csv(r'/Users/topcu/OneDrive/Desktop/BX-CSV-Dump/BX-Book-Ratings.csv',low_memory=False, sep=';', encoding='latin-1')
df_books = pd.read_csv(r'/Users/topcu/OneDrive/Desktop/BX-CSV-Dump/BX-Books.csv',low_memory=False, sep=';', encoding='latin-1',on_bad_lines='skip')
df_users = pd.read_csv(r'/Users/topcu/OneDrive/Desktop/BX-CSV-Dump/BX-Users.csv',low_memory=False, sep=';', encoding='latin-1')

**df isimli data frame'in içerisinde hiç rating verilmemiş kitapları ve 20 den az yorumlanmış kitapları listeden çıkarttık. Sonrasında ise 10 dan fazla puan veren kişileri listemizde bıraktık. En son elimizde kalan listeyi de csv dosyası olarak kaydettik**

In [4]:
df = df_books_ratings

# this combination worked pretty well 
df = df[df['Book-Rating'] != 0]

x = df['ISBN'].value_counts() >= 20
idx = x[x].index
ratings = df[df['ISBN'].isin(idx)]

x = ratings['User-ID'].value_counts() >= 10
idx = x[x].index
df_book_ratings_final = ratings[ratings['User-ID'].isin(idx)]
df_book_ratings_final.shape

df_book_ratings_final.to_csv('BX-Book-Ratings-Subset.csv', index=False, sep=';')

**Yapılan herhangi bir işleme karşılık kopyasını aldık**

In [5]:
df_book_ratings_final_copy=df_book_ratings_final.copy()

**User-ID, ISBN ve Book-Rating bilgilerini içeren tablomuz ile elimizdeki kitapları ISBN numarasına göre grupladıktan sonra her bir kitabın kaç adet rating aldığını count ettik. Sonrasında Çoktan aza doğru sıraladık. En çok rating verilen 10 kitabı csv dosyasına kaydettik** 

In [7]:
df_book_ratings_1 = df_book_ratings_final.groupby(['ISBN'])['Book-Rating'].agg(['count']).reset_index()
df_book_ratings_1 = df_book_ratings_1.sort_values(by="count", ascending=False)

df_book_ratings_1.sort_values(by="count", ascending=False).head(10)

,ISBN,count
321,0316666343,226
711,0385504209,180
251,0312195516,131
220,0142001740,128
47,0060928336,120
1617,059035342X,119
1821,0679781587,118
1178,0446672211,113
927,043935806X,108
378,0345337662,107


**Yukarıdaki veri setini bu sefer ISBN numarasına göre gruplayıp kitaplara verilen ratinglerin ortalamasını aldık. Çoktan aza doğru sıraladık. Bu kitaplarda en yüksek rating'e sahip 10 kitabı csv dosyası olarak kaydettik.**

In [8]:
df_book_ratings_2 = df_book_ratings_final.groupby(['ISBN'])['Book-Rating'].agg(['mean']).reset_index()
df_book_ratings_2 = df_book_ratings_2.sort_values(by="mean", ascending=False)
df_book_ratings_2.sort_values(by="mean", ascending=False).head(10)

,ISBN,mean
2151,3150000017,10.000000
2159,3426029553,10.000000
1628,0618002235,9.933333
930,0439425220,9.857143
25,0060256656,9.785714
2030,0836218221,9.727273
2036,0836220889,9.727273
158,0140042520,9.666667
126,006440188X,9.666667
2118,1573221112,9.666667


**Her kitabın ortalaması ve verilen rating sayısını ortak bir dataframe de topladık. C değişkeni tüm kitaplara verilen ratinglerin ortalamasıdır. m değişkeni ise her kitabın yapılan yorum sayısının 99. çeyrekteki değeridir. Bir sonraki aşamada sadece m değişkeninden büyük olan değerleri seçerek ilerletiyoruz.
weighted_rating isimli fonksiyon ile**

In [11]:
df_book_ratings_3 = df_book_ratings_final.groupby(['ISBN'])['Book-Rating'].agg(['mean', 'count']).reset_index()

# C is the mean vote across the whole report
C = df_book_ratings_3['mean'].mean()

# m is the minimum votes required to be listed in the chart;
m = df_book_ratings_3['count'].quantile(0.99)

# filter out all qualified movies into a new DataFrame
df_book_ratings_3 = df_book_ratings_3.loc[df_book_ratings_3['count'] >= m]

def weighted_rating(x, m=m, C=C):
  v = x['count']
  R = x['mean']
  return (v/(v+m) * R) + (m/(m+v) * C)

df_book_ratings_3['weight'] = df_book_ratings_3.apply(weighted_rating, axis=1)
df_book_ratings_3 = df_book_ratings_3.sort_values('weight', ascending=False)
df_book_ratings_3= df_book_ratings_3[['ISBN', 'weight']]

In [12]:
df_book_ratings_3

,ISBN,weight
927,043935806X,8.543013
1617,059035342X,8.528571
1062,0446310786,8.449308
1821,0679781587,8.360311
711,0385504209,8.344051
220,0142001740,8.329581
251,0312195516,8.247014
321,0316666343,8.124067
1178,0446672211,8.119162
238,0156027321,8.038326


In [13]:
from itertools import permutations 

def create_combinations(x):
  combinations = pd.DataFrame(list(permutations(x.values, 2)), columns=['book_a','book_b'])    
  return combinations

# use the function to create combinations
book_combinations = df_book_ratings_final.groupby('User-ID')['ISBN'].apply(create_combinations) 

book_combinations = book_combinations.reset_index(drop=True)

# count the combinations
combinations_counts = book_combinations.groupby(['book_a', 'book_b']).size()
combinations_counts = combinations_counts.to_frame(name = 'count').reset_index()

In [19]:
combinations_counts = combinations_counts.sort_values('count', ascending=False)

# only select top 10 per combination
df1 = combinations_counts.sort_values('count', ascending = False).groupby('book_a')

In [20]:
df1.head(10)

,book_a,book_b,count
769475,0439136350,0439064864,56
766448,0439064864,0439136350,56
1352136,0590353403,0439064864,51
766962,0439064864,0590353403,51
772266,0439139597,0439136350,50
...,...,...,...
646544,0399146113,0425191184,1
643876,0399145885,0671027646,1
643864,0399145885,0517122707,1
643874,0399145885,0671004107,1


In [21]:
df_reviews = df_book_ratings_final.groupby('User-ID')['ISBN'].apply(list)
reviewed = df_reviews.values.tolist()

In [22]:
# this code originated from the book Practical Recommender System. 
# Some minor tweaks to make it work with the current dataset.

from collections import defaultdict
from itertools import combinations
from datetime import datetime

def calculate_itemsets_one(reviewed, min_sup=0.01):
    N = len(reviewed)
    print(N)
    temp = defaultdict(int)
    one_itemsets = dict()

    for items in reviewed:
        for item in items:
            inx = frozenset({item})
            temp[inx] += 1

    print("temp:")
    i = 0
    # remove all items that is not supported.
    for key, itemset in temp.items():
        #print(f"{key}, {itemset}, {min_sup}, {min_sup * N}")
        if itemset > min_sup * N:
            i = i + 1
            one_itemsets[key] = itemset
    print(i)
    return one_itemsets

def calculate_itemsets_two(reviewed, one_itemsets):
    two_itemsets = defaultdict(int)

    for items in reviewed:
        items = list(set(items))  # remove duplications

        if (len(items) > 2):
            for perm in combinations(items, 2):
                if has_support(perm, one_itemsets):
                    two_itemsets[frozenset(perm)] += 1
        elif len(items) == 2:
            if has_support(items, one_itemsets):
                two_itemsets[frozenset(items)] += 1
    return two_itemsets

def calculate_association_rules(one_itemsets, two_itemsets, N):
    timestamp = datetime.now()

    rules = []
    for source, source_freq in one_itemsets.items():
        for key, group_freq in two_itemsets.items():
            if source.issubset(key):
                target = key.difference(source)
                support = group_freq / N
                confidence = group_freq / source_freq
                rules.append((timestamp, next(iter(source)), next(iter(target)),
                              confidence, support))
    return rules

def has_support(perm, one_itemsets):
  return frozenset({perm[0]}) in one_itemsets and \
    frozenset({perm[1]}) in one_itemsets

min_sup = 0.01
N = len(reviewed)

In [23]:
one_itemsets = calculate_itemsets_one(reviewed, min_sup)
two_itemsets = calculate_itemsets_two(reviewed, one_itemsets)
rules = calculate_association_rules(one_itemsets, two_itemsets, N)

# check how many associations are made
len(rules)

1834
temp:
724


353926

In [24]:
one_itemsets

{frozenset({'0060915544'}): 52,
 frozenset({'0060977493'}): 38,
 frozenset({'0316601950'}): 106,
 frozenset({'0316776963'}): 65,
 frozenset({'0316899984'}): 37,
 frozenset({'0375400117'}): 23,
 frozenset({'0385720106'}): 47,
 frozenset({'0425163407'}): 39,
 frozenset({'044023722X'}): 81,
 frozenset({'0446364800'}): 22,
 frozenset({'155874262X'}): 33,
 frozenset({'0064471047'}): 25,
 frozenset({'0142001740'}): 128,
 frozenset({'0380789035'}): 49,
 frozenset({'0399146431'}): 21,
 frozenset({'0439064864'}): 79,
 frozenset({'0439064872'}): 72,
 frozenset({'0439136350'}): 81,
 frozenset({'0439136369'}): 61,
 frozenset({'0439139597'}): 76,
 frozenset({'0451167317'}): 41,
 frozenset({'0451524934'}): 47,
 frozenset({'0451526341'}): 41,
 frozenset({'0590353403'}): 70,
 frozenset({'0618002219'}): 21,
 frozenset({'0618002227'}): 28,
 frozenset({'0671021001'}): 81,
 frozenset({'0679781587'}): 118,
 frozenset({'0316603287'}): 26,
 frozenset({'0316666343'}): 226,
 frozenset({'0316693006'}): 28,
 fro

In [25]:
two_itemsets

defaultdict(int,
            {frozenset({'0060915544', '0060977493'}): 5,
             frozenset({'0060977493', '0446364800'}): 1,
             frozenset({'0060977493', '0316601950'}): 5,
             frozenset({'0060977493', '0375400117'}): 1,
             frozenset({'0060977493', '0316776963'}): 1,
             frozenset({'0060977493', '0316899984'}): 2,
             frozenset({'0060977493', '044023722X'}): 3,
             frozenset({'0060977493', '0425163407'}): 2,
             frozenset({'0060977493', '155874262X'}): 1,
             frozenset({'0060977493', '0385720106'}): 2,
             frozenset({'0060915544', '0446364800'}): 1,
             frozenset({'0060915544', '0316601950'}): 7,
             frozenset({'0060915544', '0375400117'}): 1,
             frozenset({'0060915544', '0316776963'}): 4,
             frozenset({'0060915544', '0316899984'}): 3,
             frozenset({'0060915544', '044023722X'}): 2,
             frozenset({'0060915544', '0425163407'}): 3,
             f

In [26]:
rules

[(datetime.datetime(2022, 3, 1, 22, 49, 11, 835048),
  '0060915544',
  '0060977493',
  0.09615384615384616,
  0.0027262813522355507),
 (datetime.datetime(2022, 3, 1, 22, 49, 11, 835048),
  '0060915544',
  '0446364800',
  0.019230769230769232,
  0.0005452562704471102),
 (datetime.datetime(2022, 3, 1, 22, 49, 11, 835048),
  '0060915544',
  '0316601950',
  0.1346153846153846,
  0.003816793893129771),
 (datetime.datetime(2022, 3, 1, 22, 49, 11, 835048),
  '0060915544',
  '0375400117',
  0.019230769230769232,
  0.0005452562704471102),
 (datetime.datetime(2022, 3, 1, 22, 49, 11, 835048),
  '0060915544',
  '0316776963',
  0.07692307692307693,
  0.0021810250817884407),
 (datetime.datetime(2022, 3, 1, 22, 49, 11, 835048),
  '0060915544',
  '0316899984',
  0.057692307692307696,
  0.0016357688113413304),
 (datetime.datetime(2022, 3, 1, 22, 49, 11, 835048),
  '0060915544',
  '044023722X',
  0.038461538461538464,
  0.0010905125408942203),
 (datetime.datetime(2022, 3, 1, 22, 49, 11, 835048),
  '0060

In [117]:
associations = []

# iterate through results and create data structure containing the results
for rule in rules:
  association = {
    'source':str(rule[1]),
    'target':str(rule[2]),
    'confidence':rule[3],
    'support':rule[4]
  }
  # append to list
  associations.append(association)

# create dataframe
df_associations = pd.DataFrame(associations) 

In [28]:
df_merge = df_books.loc[:,["ISBN","Book-Title"]]

In [29]:
df_merge.head()

,ISBN,Book-Title
0,0195153448,Classical Mythology
1,0002005018,Clara Callan
2,0060973129,Decision in Normandy
3,0374157065,Flu: The Story of the Great Influenza Pandemic...
4,0393045218,The Mummies of Urumchi


In [119]:
df_associations.rename(columns ={'source':'ISBN'},inplace=True)

In [120]:
df_associations = df_associations.merge(df_merge, on = "ISBN")

In [121]:
df_associations.rename(columns ={'Book-Title':'Source_Book_Name', 'ISBN':'Source','target':'ISBN'},inplace=True)

In [122]:
df_associations = df_associations.merge(df_merge, on = "ISBN")

In [123]:
df_associations.rename(columns ={'ISBN':'Target','Book-Title':'Target_Book_Name'},inplace=True)

In [128]:
df_associations

,Source,Target,confidence,support,Source_Book_Name,Target_Book_Name
0,0060915544,0060977493,0.096154,0.002726,The Bean Trees,The God of Small Things
1,0316601950,0060977493,0.047170,0.002726,The Pilot's Wife : A Novel,The God of Small Things
2,0316776963,0060977493,0.015385,0.000545,Me Talk Pretty One Day,The God of Small Things
3,0316899984,0060977493,0.054054,0.001091,"River, Cross My Heart",The God of Small Things
4,0375400117,0060977493,0.043478,0.000545,Memoirs of a Geisha,The God of Small Things
...,...,...,...,...,...,...
351867,0316969680,0312306326,0.090909,0.001091,The Beach House,Visions of Sugar Plums
351868,0385336810,0312306326,0.047619,0.000545,Can You Keep a Secret?,Visions of Sugar Plums
351869,0451155750,0312306326,0.052632,0.000545,The Dead Zone,Visions of Sugar Plums
351870,0440414806,0312306326,0.074074,0.001091,Holes (Yearling Newbery),Visions of Sugar Plums


In [107]:
def book_recommender(isbn=False, name=False):
    if isbn:
        isbn_number=input("Please write your ISBN Number = ")
        recommend_book = df_associations[df_associations["Source"]==isbn_number]["Target"].iloc[0]
        recommend_name = df_associations[df_associations["Target"]==recommend_book]["Target_Book_Name"][0]
        print("The book we recommend to you based on the book you entered : ", recommend_name)
    else:
        book_name=input("Please write your book name = ")
        recommend_book = df_associations[df_associations["Source_Book_Name"]==book_name]["Target_Book_Name"].iloc[0]
        print("The book we recommend to you based on the book your entered :",recommend_book)